
Demos - Geo - Plotly and streamlit
==================================


* [Medium - How to Create a Simple GIS Map with Plotly and Streamlit](https://towardsdatascience.com/how-to-create-a-simple-gis-map-with-plotly-and-streamlit-7732d67b84e2)
  + Date: Dec. 2023
  + Author: Alan Jones
  ([Alan Jones on Medium](https://medium.com/@alan-jones),
  [Alan Jones on LinkedIn](https://www.linkedin.com/in/alan-jones-032699100/))
  + Publisher: Medium TowardsDataScience
  + [GitHub - repository with the code for that notebook](https://github.com/alanjones2/st-choropleth/tree/main)
* [GitHub - Data Engineering helpers - Material about geo-analysis with Python](https://github.com/data-engineering-helpers/ks-cheat-sheets/blob/main/programming/python/geo/README.md)
* [Demos - Geo - Plotly and streamlit (this notebook)](https://github.com/data-engineering-helpers/databricks-examples/blob/main/ipython-notebooks/demos-geo-plotly-and-streamlit.ipynb)

In [ ]:
%pip install streamlit plotly

In [ ]:
dbutils.library.restartPython()

In [ ]:
import pandas as pd # pyspark.pandas as pd
import streamlit as st
import plotly.express as px
import json


# Data
Our final dashboard will use data about CO2 emissions. This data is derived from tables
in a GitHub repository belonging to
[Our World in Data](https://ourworldindata.org/).
OWID is a great source of data and analysis which I have used many times
(_e.g._ [New Data Demonstrates that 2023 was the Hottest Summer Ever](https://medium.com/towards-data-science/new-data-demonstrates-that-2023-was-the-hottest-summer-ever-d92d500a8f01)).

The data that I have copied from OWID represents the CO2 emissions for countries since 1750.
The original data contains far more information than we need, so I have created subsets
of the data and stored them in the app. You will find them in the data folder.


In [ ]:
%sh

rm -rf /dbfs/tmp/demo-streamlit
mkdir -p /dbfs/tmp/demo-streamlit/{data,geo}
curl -kL https://github.com/alanjones2/st-choropleth/raw/main/data/co2_total.csv -o /dbfs/tmp/demo-streamlit/data/co2_total.csv
curl -kL https://github.com/alanjones2/st-choropleth/raw/main/data/Australian%20Bureau%20of%20Statistics.csv -o /dbfs/tmp/demo-streamlit/data/Australian-Bureau-of-Statistics.csv
head -5 /dbfs/tmp/demo-streamlit/data/co2_total.csv
curl -kL https://github.com/alanjones2/st-choropleth/raw/main/geo/australia.geojson -o /dbfs/tmp/demo-streamlit/geo/australia.geojson
ls -lFhR /dbfs/tmp/demo-streamlit


In [ ]:
df_total = pd.read_csv("/dbfs/tmp/demo-streamlit/data/co2_total.csv")
df_total_2021 = df_total[df_total['Year']==2021]

In [ ]:
col = 'Annual CO₂ emissions'
max = df_total_2021[col].max()
min = df_total_2021[col].min()

In [ ]:
fig = px.choropleth(df_total_2021, 
                    locations="Code",
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max)
                    )

In [ ]:
fig = px.choropleth(df_total_2021, 
                    locations="Code",
                    scope = 'europe',
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max),
                    title = 'Europe'
                    )

In [ ]:
st.title("Population of Australian States")
st.info("Hover over the map to see the names of the states and their population")
f = open('/dbfs/tmp/demo-streamlit/geo/australia.geojson')
oz = json.load(f)

df = pd.read_csv('/dbfs/tmp/demo-streamlit/data/Australian-Bureau-of-Statistics.csv')
fig = px.choropleth(df, geojson=oz, 
                    color="Population at 31 March 2023 ('000)",
                    locations="State", 
                    featureidkey="properties.name",
                    color_continuous_scale="Reds",
                    range_color=(0, 10000),
                    fitbounds = 'geojson',
                    template = 'plotly_white'
                   )
st.plotly_chart(fig)

In [ ]:
df = pd.read_csv('data/europop.csv')

fig = px.scatter_geo(df, scope = 'europe', 
                    color="Population (historical estimates)",
                    size="Population (historical estimates)",
                    locations="Code", 
                    hover_name = 'Entity',
                    color_continuous_scale="Purples",
                    range_color=(0, 100000000),
                    fitbounds = 'locations',
                    template = 'plotly_white',
                    title = "European populations"
                   )

st.plotly_chart(fig)

In [ ]:
fig = px.choropleth(df_total[df_total['Year']==year], 
                    locations="Code",
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max),
                    scope= 'world',
                    projection="orthographic",
                    color_continuous_scale=px.colors.sequential.Reds)
st.plotly_chart(fig)

In [ ]:
st.set_page_config(layout="wide")

st.title("CO2 Emissions")
st.write("""The following maps display the CO2 emissions for a
            range of countries over a range of time""")

st.info("""Use the slider to select a year to display
           the total emissions for each country. 
           Scroll down to see an interactive 3D representation.""")

col1, col2 = st.columns(2)

df_total = pd.read_csv('data/co2_total.csv')
col = 'Annual CO₂ emissions'
max = df_total[col].max()
min = df_total[col].min()

# To get the whole range replace 1950 with the comment that follows it
first_year = 1950 #df_total['Year'].min()
last_year = df_total['Year'].max()
year = st.slider('Select year',first_year,last_year, key=col)

col1.write("""This map uses the 'Natural Earth' projection""")
fig = px.choropleth(df_total[df_total['Year']==year], 
                    locations="Code",
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max),
                    scope= 'world',
                    projection="natural earth",
                    color_continuous_scale=px.colors.sequential.Reds)
col1.plotly_chart(fig)

col2.write("""This map uses the 'Orthographic' projection.
Click on the globe and move the pointer to rotate it.
""")

fig = px.choropleth(df_total[df_total['Year']==year], locations="Code",
                    color=col,
                    hover_name="Entity",
                    range_color=(min,max),
                    scope= 'world',
                    projection="orthographic",
                    color_continuous_scale=px.colors.sequential.Reds)
col2.plotly_chart(fig)

In [ ]:
# add/subtract from the selected countries
    c = st.multiselect('Add a country:', countries, default=['United States', 'China', 'Russia', 'Germany'])
    tab1, tab2 = col2.tabs(["Graph", "Table"])

    with tab1:
        # plot a line graph of emissions for selected countries
        fig = px.line(df_total[df_total['Entity'].isin(c)], x='Year', y='Annual CO₂ emissions', color = 'Entity')
        st.plotly_chart(fig, use_container_width=True)
    with tab2:
        table = df_total[df_total['Year']==st.session_state['year']]
        st.dataframe(table[table['Entity'].isin(c)], use_container_width=True)